In [9]:
import pandas as pd

df = pd.read_csv('BTC-USD_17.09.2014-14.03.2024.csv')

df['Date'] = pd.to_datetime(df['Date'], format='%d-%m-%y')

In [10]:
from datetime import datetime

def prompt_for_date(prompt_message, min_date=None, max_date=None, comparison_date=None, must_be_after=False):
    while True:
        date_input = input(prompt_message)
        try:
            date = datetime.strptime(date_input, "%d-%m-%Y")
            if min_date is not None and date < min_date:
                print("The date must not be before %s." % min_date.strftime("%d-%m-%Y"))
                continue
            if max_date is not None and date > max_date:
                print("The date must not be after %s." % max_date.strftime("%d-%m-%Y"))
                continue
            if comparison_date is not None:
                if must_be_after and date <= comparison_date:
                    print("The end date must be after the start date %s." % comparison_date.strftime("%d-%m-%Y"))
                    continue
            return date
        except ValueError:
            print("Invalid date format. Please use the format dd-mm-yyyy.")

min_allowed_start_date = datetime.strptime("17-09-2014", "%d-%m-%Y")
max_allowed_end_date = datetime.strptime("14-03-2024", "%d-%m-%Y")

start_date = prompt_for_date("Enter the start date (dd-mm-yyyy).", min_date=min_allowed_start_date)
end_date = prompt_for_date("Enter the end date (dd-mm-yyyy).", max_date=max_allowed_end_date, comparison_date=start_date, must_be_after=True)

def prompt_for_number(prompt_message, minimum=None, condition=lambda x: True):
    while True:
        try:
            number = float(input(prompt_message))
            if minimum is not None and number <= minimum:
                print("The value must be greater than %s." % minimum)
                continue
            if not condition(number):
                print("The input does not meet the required condition.")
                continue
            return number
        except ValueError:
            print("Please enter a valid number.")

initial_balance = prompt_for_number("Enter the initial balance of the trading bot wallet: ", minimum=0)
profit_percent = prompt_for_number("Enter the % profit (value must be >= 1): ", minimum=1.00) 

start_date_df_ascending_6month = pd.to_datetime('2020-10-01')
end_date_df_ascending_6month  = pd.to_datetime('2021-04-01')
start_date_df_descending_6month = pd.to_datetime('2021-10-01')
end_date_df_descending_6month  = pd.to_datetime('2022-09-01')
start_date_df_stagnation_6month = pd.to_datetime('2015-11-01')
end_date_df_stagnation_6month  = pd.to_datetime('2016-04-01')
start_date_df_combined_1year = pd.to_datetime('2021-10-01')
end_date_df_combined_1year  = pd.to_datetime('2022-11-01')

In [11]:
def prepare_and_export(df, file_name, initial_balance, initial_profit):
    df.rename(columns={'Date': 'date', 'Open': 'open', 'High': 'high', 'Low': 'low', 'Close': 'close', 'Volume': 'volume'}, inplace=True)
    df.insert(1, 'time', '00:00')
    df['date'] = df['date'].dt.strftime('%m/%d/%Y')
    df['date'] = pd.to_datetime(df['date'])
    df.drop(columns=['Adj Close'], inplace=True)
    df['initial_balance'] = initial_balance
    df['profit'] = initial_profit
    df.to_csv(file_name, index=False)

filtered_df = df[(df['Date'] >= start_date) & (df['Date'] <= end_date)].copy()
filtered_df_ascending_6month = df[(df['Date'] >= start_date_df_ascending_6month) & (df['Date'] <= end_date_df_ascending_6month)].copy()
filtered_df_descending_6month = df[(df['Date'] >= start_date_df_descending_6month) & (df['Date'] <= end_date_df_descending_6month)].copy()
filtered_df_stagnation_6month = df[(df['Date'] >= start_date_df_stagnation_6month) & (df['Date'] <= end_date_df_stagnation_6month)].copy()
filtered_df_combined_1year = df[(df['Date'] >= start_date_df_combined_1year) & (df['Date'] <= end_date_df_combined_1year)].copy()

prepare_and_export(filtered_df, 'btcusd_tb1_data.csv', initial_balance, profit_percent)
prepare_and_export(filtered_df_ascending_6month, 'btcusd_tb1_asc_data.csv', initial_balance, profit_percent)
prepare_and_export(filtered_df_descending_6month, 'btcusd_tb1_dsc_data.csv', initial_balance, profit_percent)
prepare_and_export(filtered_df_stagnation_6month, 'btcusd_tb1_stg_data.csv', initial_balance, profit_percent)
prepare_and_export(filtered_df_combined_1year, 'btcusd_tb1_com_data.csv', initial_balance, profit_percent)

In [12]:
import time
from datetime import datetime

def trading_bot_1(balance, data, fee_percent, profit_percent):
    trades = []
    holding = 0
    last_buy_balance = 0

    for index, row in data.iterrows():
        date = row['date']
        unix_date = int(time.mktime(date.timetuple()))
        open_price, high_price, low_price, close_price = row['open'], row['high'], row['low'], row['close']

        if holding == 0:
            buy_price = open_price
            fee = fee_percent * balance / 100
            real_balance = balance - fee
            last_buy_balance = real_balance
            holding = real_balance / buy_price
            trades.append([unix_date, date, 'BUY', buy_price, open_price, high_price, low_price, close_price, real_balance])
        if holding > 0:
            sell_value = holding * close_price
            profit = (sell_value - last_buy_balance) / last_buy_balance * 100

            if profit >= profit_percent:
                fee = fee_percent * sell_value / 100
                balance = sell_value - fee
                holding = 0
                last_buy_balance =  sell_value
                trades.append([unix_date, date, 'SELL', close_price, open_price, high_price, low_price, close_price, balance])
                
    final_balance = trades[-1][-1]
    return final_balance, trades

In [13]:
print(f"Initial Balance: ${initial_balance}")

final_balance, trades = trading_bot_1(initial_balance, filtered_df, 1, profit_percent)
print(f"Final Balance Custom Data: ${final_balance}")
trades_df = pd.DataFrame(trades, columns=['unix', 'date', 'trade', 'price', 'open', 'high', 'low', 'close', 'balance'])
trades_df['date'] = pd.to_datetime(trades_df['date'])
trades_df['date'] = trades_df['date'].apply(lambda x: x.strftime('%Y-%m-%d %H:%M:%S'))
trades_df.to_csv('btcusd_buy_tb1_trades.csv', index=False)

final_balance_1, trades_asc = trading_bot_1(initial_balance, filtered_df_ascending_6month, 1, profit_percent)
print(f"Final Balance ASC_6M: ${final_balance_1}")
trades_df = pd.DataFrame(trades_asc, columns=['unix', 'date', 'trade', 'price', 'open', 'high', 'low', 'close', 'balance'])
trades_df['date'] = pd.to_datetime(trades_df['date'])
trades_df['date'] = trades_df['date'].apply(lambda x: x.strftime('%Y-%m-%d %H:%M:%S'))
trades_df.to_csv('btcusd_buy_tb1_asc_trades.csv', index=False)

final_balance_2, trades_dsc = trading_bot_1(initial_balance, filtered_df_descending_6month, 1, profit_percent)
print(f"Final Balance DSC_6M: ${final_balance_2}")
trades_df = pd.DataFrame(trades_dsc, columns=['unix', 'date', 'trade', 'price', 'open', 'high', 'low', 'close', 'balance'])
trades_df['date'] = pd.to_datetime(trades_df['date'])
trades_df['date'] = trades_df['date'].apply(lambda x: x.strftime('%Y-%m-%d %H:%M:%S'))
trades_df.to_csv('btcusd_buy_tb1_dsc_trades.csv', index=False)

final_balance_3, trades_stg = trading_bot_1(initial_balance, filtered_df_stagnation_6month, 1, profit_percent)
print(f"Final Balance STG_6M: ${final_balance_3}")
trades_df = pd.DataFrame(trades_stg, columns=['unix', 'date', 'trade', 'price', 'open', 'high', 'low', 'close', 'balance'])
trades_df['date'] = pd.to_datetime(trades_df['date'])
trades_df['date'] = trades_df['date'].apply(lambda x: x.strftime('%Y-%m-%d %H:%M:%S'))
trades_df.to_csv('btcusd_buy_tb1_stg_trades.csv', index=False)

final_balance_4, trades_com = trading_bot_1(initial_balance, filtered_df_combined_1year, 1, profit_percent)
print(f"Final Balance COM_1Y: ${final_balance_4}")
trades_df = pd.DataFrame(trades_com, columns=['unix', 'date', 'trade', 'price', 'open', 'high', 'low', 'close', 'balance'])
trades_df['date'] = pd.to_datetime(trades_df['date'])
trades_df['date'] = trades_df['date'].apply(lambda x: x.strftime('%Y-%m-%d %H:%M:%S'))
trades_df.to_csv('btcusd_buy_tb1_com_trades.csv', index=False)

Initial Balance: $100.0
Final Balance Custom Data: $2150.1552464317906
Final Balance ASC_6M: $278.95594844665595
Final Balance DSC_6M: $127.11683929985772
Final Balance STG_6M: $129.33517844435576
Final Balance COM_1Y: $127.11683929985772
